In [5]:
# data collection
import cv2
import os,sys
import numpy as np
import csv

name = input("Enter name")
choice  = input("1:Images from camera\n2:images from gallery\n")
detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

Id = 0

if os.path.exists("SubjectDetails.csv"):
    num_rows= 0
    for row in open("SubjectDetails.csv"):
        print(num_rows)
        num_rows += 1
    Id = num_rows
else:
    Id = "1"
    
if choice == "1":
    # body to get images from camera
    camera = cv2.VideoCapture(0)
    sample_num = 0
    while True:
        ret,frame = camera.read()
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        faces = detector.detectMultiScale(gray,1.2,5)
        if faces is not ():
            for (x,y,w,h) in faces:
                cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),1)
                cv2.putText(frame,str(sample_num),(10,90),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),1)
                sample_num += 1
                cv2.imwrite("subject_images/"+name +"."+str(Id) +'.'+str(sample_num) + ".jpg",gray[y:y+h,x:x+w])
        cv2.imshow("frame",frame)
        if cv2.waitKey(100) & 0xFF == 27:
            break
    camera.release()
    cv2.destroyAllWindows()
    
elif choice == "2":
    path = "steve jobs/"
    images_paths = [os.path.join(path,f) for f in os.listdir(path)]
    sample_num = 0
    for image in images_paths:
        image = cv2.imread(image)
        gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        faces = detector.detectMultiScale(gray,1.2,5)
        if faces is not ():
            for (x,y,w,h) in faces:
                sample_num += 1
                cv2.imwrite("subject_images/"+name +"."+str(Id) +'.'+str(sample_num) + ".jpg",gray[y:y+h,x:x+w])        
else:
    print("invalid value")
    
row = [name,Id]  
if os.path.exists("SubjectDetails.csv"):
    with open("SubjectDetails.csv",'a+',newline = '') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(row)
        csvfile.close()
else:
    with open("SubjectDetails.csv",'w',newline = '') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['name','Id'])
        writer.writerow(row)
        csvfile.close()

Enter name steve job
1:Images from camera
2:images from gallery
 2


0
1
